---
## <span style="color:orange"> Inside account B (centralized feature store)</span>
---

<div style="text-align: justify">Run this notebook within account B. This must be the account where the centralized feature store exists.
    In this notebook, we create a feature group named <b>customers</b> which is a collection of customer profile features and populate the features. The features are written to the Online and Offline stores of the centralized feature store.
Later, we show, 1/ how to read the features from the Online store and 2/ how to read the features from the Offline store via Athena query to create a training set.</div>

**IMPORTANT:** This notebook must run be run BEFORE you had executed notebook `account-a`.

### Imports 

In [3]:
import sagemaker
import logging
import pandas
import boto3
import json
import time

#### Setup logging

In [4]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.INFO)
logger.addHandler(logging.StreamHandler())
logger.info(f'[Using Boto3 version: {boto3.__version__}]')

[Using Boto3 version: 1.16.37]


### Essentials

In [ ]:
OFFLINE_STORE_BUCKET = '<NAME OF OFFLINE STORE S3 BUCKET IN THIS ACCOUNT>'
ACCOUNT_ID = '<ACCOUNT B ID>'
REGION = '<AWS REGION>'
FEATURE_GROUP_NAME = 'customers'

In [ ]:
role_arn = sagemaker.get_execution_role()

In [ ]:
sagemaker_client = boto3.client('sagemaker')
sagemaker_featurestore_runtime_client = boto3.client(service_name='sagemaker-featurestore-runtime')
s3_client = boto3.client(service_name='s3')
athena_client = boto3.client(service_name='athena')

In [ ]:
offline_config = {'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': f's3://{OFFLINE_STORE_BUCKET}'}}}
# offline_config = {}  # uncomment and use this line if needed to write ONLY to the Online feature store

### Load schema

In [ ]:
def load_schema(schema):
    feature_definitions = []
    for col in schema['features']:
        feature = {'FeatureName': col['name']}
        if col['type'] == 'double':
            feature['FeatureType'] = 'Fractional'
        elif col['type'] == 'bigint':
            feature['FeatureType'] = 'Integral'
        else:
            feature['FeatureType'] = 'String'
        feature_definitions.append(feature)
    return feature_definitions, schema['record_identifier_feature_name'], schema['event_time_feature_name']

In [ ]:
schema = json.loads(open('./schema/customers.json').read())
feature_definitions, record_identifier_feature_name, event_time_feature_name = load_schema(schema)

In [ ]:
feature_definitions

### Create a feature group

Uncomment and run the cell below if the feature group already exists or during re-runs.

In [ ]:
# sagemaker_client.delete_feature_group(FeatureGroupName=FEATURE_GROUP_NAME)

In [ ]:
sagemaker_client.create_feature_group(FeatureGroupName=FEATURE_GROUP_NAME,
                                    RecordIdentifierFeatureName=record_identifier_feature_name,
                                    EventTimeFeatureName=event_time_feature_name,
                                    FeatureDefinitions=feature_definitions,
                                    Description=schema['description'],
                                    Tags=schema['tags'],
                                    OnlineStoreConfig={'EnableOnlineStore': True},
                                    RoleArn=role_arn,
                                    **offline_config)

In [ ]:
sagemaker_client.describe_feature_group(FeatureGroupName=FEATURE_GROUP_NAME)

### Populate features to the feature group

In [5]:
customers_df = pandas.read_csv('./data/customers.csv', header=None)
customers_df

,0,1,2,3,4,5,6
0,1001,aaron,32,married,male,dublin,OH
1,1002,bria,18,single,female,santa barbara,CA
2,1003,cindy,24,married,female,dallas,TX
3,1004,daniel,24,single,male,miami,FL
4,1005,emily,36,single,female,portland,OR


In [6]:
records = []
for _, row in customers_df.iterrows():
    cid, name, age, marital_status, sex, city, state = row
    record = []
    record.append({'ValueAsString': str(cid), 'FeatureName': 'cid'})
    record.append({'ValueAsString': name, 'FeatureName': 'name'})
    record.append({'ValueAsString': str(age), 'FeatureName': 'age'})
    record.append({'ValueAsString': marital_status, 'FeatureName': 'marital_status'})
    record.append({'ValueAsString': sex, 'FeatureName': 'sex'})
    record.append({'ValueAsString': city, 'FeatureName': 'city'})
    record.append({'ValueAsString': state, 'FeatureName': 'state'})
    event_time_feature = {'ValueAsString': str(int(round(time.time()))), 'FeatureName': 'created_at'}
    record.append(event_time_feature)
    records.append(record)

In [7]:
records[0]

[{'ValueAsString': '1001', 'FeatureName': 'cid'},
 {'ValueAsString': 'aaron', 'FeatureName': 'name'},
 {'ValueAsString': '32', 'FeatureName': 'age'},
 {'ValueAsString': 'married', 'FeatureName': 'marital_status'},
 {'ValueAsString': 'male', 'FeatureName': 'sex'},
 {'ValueAsString': 'dublin', 'FeatureName': 'city'},
 {'ValueAsString': 'OH', 'FeatureName': 'state'},
 {'ValueAsString': '1614966953', 'FeatureName': 'created_at'}]

#### Write features to the feature store

In [ ]:
for record in records:
    response = sagemaker_featurestore_runtime_client.put_record(FeatureGroupName=FEATURE_GROUP_NAME, 
                                                                Record=record)
    print(response['ResponseMetadata']['HTTPStatusCode'])

#### Verify if we can retrieve features from the `customers` feature group using record identifier
Here, we are reading features from the Online store.

In [ ]:
response = sagemaker_featurestore_runtime_client.get_record(FeatureGroupName=FEATURE_GROUP_NAME, 
                                                            RecordIdentifierValueAsString='1002'
                                                           )
response

### Get records from the Offline store (S3 bucket)
Now let us wait for the data to appear in the Offline store (S3 bucket) before moving forward to creating a dataset. This will take approximately take <= 5 minutes.

In [ ]:
feature_group_s3_prefix = f'{ACCOUNT_ID}/sagemaker/{REGION}/offline-store/{FEATURE_GROUP_NAME}/data'
feature_group_s3_prefix

In [ ]:
offline_store_contents = None
while offline_store_contents is None:
    objects = s3_client.list_objects(Bucket=OFFLINE_STORE_BUCKET, Prefix=feature_group_s3_prefix)
    if 'Contents' in objects and len(objects['Contents']) > 1:
        logger.info('[Features are available in Offline Store!]')
        offline_store_contents = objects['Contents']
    else:
        logger.info('[Waiting for data in Offline Store...]')
        time.sleep(60)

### Use Athena to query features from the Offline store and create training set

In [ ]:
feature_group = sagemaker_client.describe_feature_group(FeatureGroupName=FEATURE_GROUP_NAME)

In [ ]:
glue_table_name = feature_group['OfflineStoreConfig']['DataCatalogConfig']['TableName']

In [ ]:
query_string = f'SELECT * FROM "{glue_table_name}"'
query_string

#### Run Athena query and save results 

You can save the results of the Athena query to a folder within the Offline store S3 bucket or another bucket.

In [ ]:
response = athena_client.start_query_execution(
                QueryString=query_string,
                QueryExecutionContext={
                    'Database': 'sagemaker_featurestore',
                    'Catalog': 'AwsDataCatalog'
                },
                ResultConfiguration={
                    'OutputLocation': f's3://{OFFLINE_STORE_BUCKET}/query_results/customers',
                }
            )

In [ ]:
query_results = athena_client.get_query_results(QueryExecutionId=response['QueryExecutionId'],
                                                MaxResults=100)

In [ ]:
training_set_csv_s3_key = None
for s3_object in s3_client.list_objects(Bucket=OFFLINE_STORE_BUCKET)['Contents']:
    key = s3_object['Key']
    if key.startswith('query_results/customers') and key.endswith('csv'):
        training_set_csv_s3_key = key

In [ ]:
training_set_s3_path = f's3://{OFFLINE_STORE_BUCKET}/{training_set_csv_s3_key}'
training_set_s3_path

In [ ]:
training_set = pandas.read_csv(training_set_s3_path)

In [ ]:
training_set